In [71]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.svm import SVC
from collections import Counter

from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.preprocessing import sequence
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer


In [72]:
from nltk import word_tokenize
from collections import defaultdict

def count_top_x_words(corpus, top_x, skip_top_n):
    count = defaultdict(lambda: 0)
    for c in corpus:
        for w in word_tokenize(c):
            count[w] += 1
    count_tuples = sorted([(w, c) for w, c in count.items()], key=lambda x: x[1], reverse=True)
    return [i[0] for i in count_tuples[skip_top_n: skip_top_n + top_x]]


def replace_top_x_words_with_vectors(corpus, top_x):
    topx_dict = {top_x[i]: i for i in range(len(top_x))}

    return [
        [topx_dict[w] for w in word_tokenize(s) if w in topx_dict]
        for s in corpus
    ], topx_dict


def filter_to_top_x(corpus, n_top, skip_n_top=0):
    top_x = count_top_x_words(corpus, n_top, skip_n_top)
    return replace_top_x_words_with_vectors(corpus, top_x)

In [73]:
df = pd.read_csv("filteredData.csv")

In [74]:
# counter = Counter(df['bias'].tolist())
content_list = df['content'].tolist()
bias_List = df['bias'].tolist()

In [75]:
mapped_list, word_list = filter_to_top_x(content_list, 2500, 0)
# mapped_list = sequence.pad_sequences(mapped_list, maxlen=500)



In [83]:
from keras.utils import pad_sequences

In [84]:
mapped_list = pad_sequences(mapped_list, maxlen=500)
# preprocessing.sequence.pad_sequences(mapped_list, maxlen=500)


In [85]:
train_x, test_x, train_y, test_y = train_test_split(mapped_list, bias_List, test_size=0.3)


In [86]:
clf = SVC(kernel='linear').fit(train_x, train_y)
y_score = clf.predict(test_x)

In [39]:
n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 61.99%
